# DG - Methods for elliptic problems

In [ ]:
from ngsolve import *
from netgen.geom2d import unit_square
from ngsolve.webgui import Draw
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

In [ ]:
order = 3
fes = L2(mesh, order=order, dgjumps=True)

In [ ]:
u,v = fes.TnT()

h = specialcf.mesh_size
n = specialcf.normal(2)
alpha = 3
dS = dx(skeleton=True)   # internal edges

def jump(u): return u-u.Other()
def mean(u): return 0.5*(u+u.Other())

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx 
a += (-n*mean(grad(u))*jump(v)-n*mean(grad(v))*jump(u))*dS
a += alpha*(order+1)**2/h*jump(u)*jump(v)*dS

a += alpha*order**2/h*u*v * ds(skeleton=True) 
a += (-n*grad(u)*v-n*grad(v)*u)* ds(skeleton=True)

f = LinearForm(fes)
f += 10*v*dx

a.Assemble()
f.Assemble()

inv = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")

print ("ndof: ", fes.ndof)
print ("non-zero(A):", a.mat.nze)
print ("non-zero(Inv):", inv.nze)

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = inv * f.vec

In [ ]:
Draw (gfu);